In [1]:
pip install pyspark

In [2]:
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.5.3/spark-3.5.3-bin-hadoop3.tgz
!tar -xvzf spark-3.5.3-bin-hadoop3.tgz
!pip install -q findspark

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,535 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [3,000 kB]
Get:13 https://cloud.r-project.org/bin/linux/ubuntu jammy-

In [3]:
import os
import findspark

# Set Spark & Java paths
os.environ["SPARK_HOME"] = "/content/spark-3.5.3-bin-hadoop3"
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"

# Initialize findspark
findspark.init()

In [4]:
from pyspark.sql import SparkSession

# Create a Spark session
spark = SparkSession.builder.appName("TestSpark").getOrCreate()

# Check if Spark is working
print(spark.version)  # Should print "3.5.3"

3.5.3


In [9]:
from pyspark import SparkFiles

# Define S3 URL
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

# Add file to Spark
spark.sparkContext.addFile(url)

# Read CSV into DataFrame
home_sales_df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)

# Show the DataFrame
home_sales_df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [10]:
# 2. Create a temporary view of the DataFrame
home_sales_df.createOrReplaceTempView("home_sales_view")

# Now you can run SQL queries on the temporary view
# Example: Running a SQL query to count the number of rows
result = spark.sql("SELECT COUNT(*) AS total_rows FROM home_sales_view")

# Show the result
result.show()


+----------+
|total_rows|
+----------+
|     33287|
+----------+



In [12]:
from pyspark.sql import functions as F

# 1. Filter for four-bedroom houses
four_bedroom_houses = home_sales_df.filter(home_sales_df["bedrooms"] == "4")

# 2. Cast the 'price' column to float and 'date' column to date
four_bedroom_houses = four_bedroom_houses.withColumn("price", F.col("price").cast("float"))
four_bedroom_houses = four_bedroom_houses.withColumn("date", F.to_date(four_bedroom_houses["date"], "yyyy-MM-dd"))

# 3. Extract the year from the 'date' column
four_bedroom_houses = four_bedroom_houses.withColumn("year", F.year(four_bedroom_houses["date"]))

# 4. Group by "year" and calculate the average price, rounded to 2 decimal places
avg_price_per_year = four_bedroom_houses.groupBy("year").agg(F.round(F.avg("price"), 2).alias("avg_price"))

# 5. Show the result
avg_price_per_year.show()


+----+---------+
|year|avg_price|
+----+---------+
|2022|296363.88|
|2019| 300263.7|
|2020|298353.78|
|2021|301819.44|
+----+---------+



In [13]:
# Register the DataFrame as a temporary SQL view
home_sales_df.createOrReplaceTempView("home_sales")

# Corrected SQL query
sql = """
SELECT YEAR(date_built) AS YEAR_BUILT,
       ROUND(AVG(price), 2) AS AVERAGE_PRICE
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3
GROUP BY YEAR(date_built)
ORDER BY YEAR_BUILT DESC
"""

# Run the SQL query and show the results
spark.sql(sql).show()


+----------+-------------+
|YEAR_BUILT|AVERAGE_PRICE|
+----------+-------------+
|      2017|    292676.79|
|      2016|    290555.07|
|      2015|     288770.3|
|      2014|    290852.27|
|      2013|    295962.27|
|      2012|    293683.19|
|      2011|    291117.47|
|      2010|    292859.62|
+----------+-------------+



In [14]:
# Register the DataFrame as a temporary SQL view
home_sales_df.createOrReplaceTempView("home_sales")

# SQL query with the required conditions
sql = """
SELECT YEAR(date_built) AS YEAR_BUILT,
       ROUND(AVG(price), 2) AS AVERAGE_PRICE
FROM home_sales
WHERE bedrooms = 3
  AND bathrooms = 3
  AND floors = 2
  AND sqft_living >= 2000
GROUP BY YEAR(date_built)
ORDER BY YEAR_BUILT DESC
"""

# Run the SQL query and show the results
spark.sql(sql).show()


+----------+-------------+
|YEAR_BUILT|AVERAGE_PRICE|
+----------+-------------+
|      2017|    280317.58|
|      2016|     293965.1|
|      2015|    297609.97|
|      2014|    298264.72|
|      2013|    303676.79|
|      2012|    307539.97|
|      2011|    276553.81|
|      2010|    285010.22|
+----------+-------------+



In [16]:
import time
from pyspark.sql import SparkSession

# Assuming Spark session is already created
# spark = SparkSession.builder.appName("HomeSales").getOrCreate()

# Start the timer
start_time = time.time()

# Register the DataFrame as a temporary SQL view
home_sales_df.createOrReplaceTempView("home_sales")

# SQL query to calculate average price per "view", filter by price >= 350,000, and order by descending view rating
sql = """
SELECT view,
       ROUND(AVG(price), 2) AS average_price
FROM home_sales
GROUP BY view
HAVING AVG(price) >= 350000
ORDER BY view DESC
"""

# Run the SQL query and show the results
result = spark.sql(sql)
result.show()

# Print the execution time
print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|average_price|
+----+-------------+
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|    1063498.0|
|  81|   1053472.79|
|  80|    991767.38|
+----+-------------+
only showing top 20 rows

--- 1.8571274280548096 seconds ---


In [17]:
# Cache the DataFrame to improve query performance
home_sales_df.cache()

# Register the cached DataFrame as a temporary SQL view
home_sales_df.createOrReplaceTempView("home_sales")

# Check if the table is cached by calling the `is_cached` method
print(f"Is home_sales cached? {spark.catalog.isCached('home_sales')}")


Is home_sales cached? True


In [18]:
# Check if the table 'home_sales' is cached
spark.catalog.isCached('home_sales')


True

In [24]:

# Assuming the 'home_sales' table is already cached
# If not, you can cache it using:
# home_sales_df.cache()
# home_sales_df.createOrReplaceTempView("home_sales")

# 9. Using the cached data, run the query and measure runtime
start_time = time.time()

# SQL query to calculate average price per "view" rating, with average price >= $350,000
sql_query = """
SELECT view,
       ROUND(AVG(price), 2) AS avg_price
FROM home_sales
GROUP BY view
HAVING avg_price >= 350000
ORDER BY view DESC
"""

# Run the SQL query using the cached table
result = spark.sql(sql_query)

# Show the result
result.show()

# Calculate and print the runtime for the cached query
cached_runtime = time.time() - start_time
print(f"--- Cached query runtime: {cached_runtime} seconds ---")


+----+----------+
|view| avg_price|
+----+----------+
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
|  80| 991767.38|
+----+----------+
only showing top 20 rows

--- Cached query runtime: 0.5431132316589355 seconds ---


In [31]:
#  Assuming `home_sales_df` is already defined
# (This DataFrame should have the 'date_built' field.)

#  Partition the data by 'date_built' and write it to Parquet
output_path = "/path/to/output/home_sales_partitioned"  # You can specify your path here

# Write the DataFrame to Parquet format, partitioned by 'date_built', using overwrite mode
home_sales_df.write.partitionBy("date_built").mode("overwrite").parquet(output_path)

print(f"Data written to {output_path} with partitioning by 'date_built' and overwrite mode.")


Data written to /path/to/output/home_sales_partitioned with partitioning by 'date_built' and overwrite mode.


In [32]:
# Define the path to the partitioned Parquet data
input_path = "/path/to/output/home_sales_partitioned"  # Use the correct path from your environment

# Read the partitioned Parquet data into a DataFrame
home_sales_parquet_df = spark.read.parquet(input_path)

# Show the first few rows of the DataFrame to verify the data is loaded correctly
home_sales_parquet_df.show()


+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|                  id|      date| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|date_built|
+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|2ed8d509-7372-46d...|2021-08-06|258710|       3|        3|       1918|    9666|     1|         0|  25|      2015|
|941bad30-eb49-4a7...|2020-05-09|229896|       3|        3|       2197|    8641|     1|         0|   3|      2015|
|c797ca12-52cd-4b1...|2019-06-08|288650|       2|        3|       2100|   10419|     2|         0|   7|      2015|
|0cfe57f3-28c2-472...|2019-10-04|308313|       3|        3|       1960|    9453|     2|         0|   2|      2015|
|d715f295-2fbf-4e9...|2021-05-17|391574|       3|        2|       1635|    8040|     2|         0|  10|      2015|
|a18515a2-86f3-46b...|2022-02-18|419543|       3|        2|       1642|   12826|

In [33]:
# Create a temporary table for the parquet data
home_sales_parquet_df.createOrReplaceTempView("home_sales_parquet")

# Verify by running a simple query on the temporary table
spark.sql("SELECT * FROM home_sales_parquet LIMIT 5").show()


+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|                  id|      date| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|date_built|
+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|2ed8d509-7372-46d...|2021-08-06|258710|       3|        3|       1918|    9666|     1|         0|  25|      2015|
|941bad30-eb49-4a7...|2020-05-09|229896|       3|        3|       2197|    8641|     1|         0|   3|      2015|
|c797ca12-52cd-4b1...|2019-06-08|288650|       2|        3|       2100|   10419|     2|         0|   7|      2015|
|0cfe57f3-28c2-472...|2019-10-04|308313|       3|        3|       1960|    9453|     2|         0|   2|      2015|
|d715f295-2fbf-4e9...|2021-05-17|391574|       3|        2|       1635|    8040|     2|         0|  10|      2015|
+--------------------+----------+------+--------+---------+-----------+--------+

In [39]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()

# Run the SQL query without caching and measure runtime
start_time_uncached = time.time()

# Run the SQL query without caching
result_uncached = spark.sql(sql)
result_uncached.show()

# Print the uncached execution time
print("--- Uncached: %s seconds ---" % (time.time() - start_time_uncached))

# Now let's cache the DataFrame before running the same query again

parquet_df.cache()  # Cache the DataFrame

start_time_cached = time.time()

# Run the same SQL query with caching
result_cached = spark.sql(sql)
result_cached.show()

# Print the cached execution time
print("--- Cached: %s seconds ---" % (time.time() - start_time_cached))



print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|average_price|
+----+-------------+
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|    1063498.0|
|  81|   1053472.79|
|  80|    991767.38|
+----+-------------+
only showing top 20 rows

--- Uncached: 0.3160684108734131 seconds ---
+----+-------------+
|view|average_price|
+----+-------------+
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82

In [40]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable("home_sales")


In [41]:
# 15. Check if the home_sales is no longer cached
is_cached = spark.catalog.isCached("home_sales")

# Print the result
print(f"Is 'home_sales' table cached? {is_cached}")


Is 'home_sales' table cached? False
